In [1]:
cd ..

/Users/amlvt225/Code/GitHub/donkey_golf


In [2]:
ls

README.md          jupyter_notebooks/ run.py
donkey_golf/       requirements.txt


In [3]:
import donkey_golf as dg

/Users/amlvt225/Code/VirtualEnvs/venv_dgolf/lib/python3.7/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [4]:
import oyaml
import pandas as pd
import logging
import psycopg2

from sqlalchemy import create_engine

from donkey_golf import config

# create logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

In [5]:
dd = dg.data_things.DraftDay()

2019-04-24 22:03:12,747 - donkey_golf.data_things.base_class - DEBUG - ABOUT TO SET yaml_sql_dict
2019-04-24 22:03:12,757 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-04-24 22:03:13,029 - donkey_golf.data_things.base_class - INFO - Current tourney ID: 401056527


In [6]:
dd.run()

2019-04-24 22:03:14,381 - donkey_golf.data_things.draft_day - INFO - Pulling a list of available players for the draft
2019-04-24 22:03:14,382 - donkey_golf.data_things.base_class - INFO - Executing run_sql


In [7]:
dd.data

,player,current_rank,events_played,rank,tier
0,Dustin Johnson,1.0,47.0,1.0,Tier 1
1,Justin Rose,2.0,47.0,2.0,Tier 1
2,Brooks Koepka,3.0,44.0,3.0,Tier 1
3,Rory McIlroy,4.0,46.0,4.0,Tier 1
4,Justin Thomas,5.0,48.0,5.0,Tier 1
5,Tiger Woods,6.0,26.0,6.0,Tier 1
6,Francesco Molinari,7.0,47.0,7.0,Tier 1
7,Bryson DeChambeau,8.0,56.0,8.0,Tier 1
8,Xander Schauffele,9.0,57.0,9.0,Tier 1
9,Rickie Fowler,10.0,46.0,10.0,Tier 1


In [ ]:
print(dd.sql)

In [ ]:
lb = dg.data_things.LoadLeaderboard()

In [ ]:
lb.scrape_espn_leaderboard('http://www.espn.com/golf/leaderboard')

In [8]:
cmon = dg.data_things.PullLeaderboard(user_id=1)

2019-04-24 22:05:42,858 - donkey_golf.data_things.base_class - DEBUG - ABOUT TO SET yaml_sql_dict
2019-04-24 22:05:42,870 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-04-24 22:05:43,167 - donkey_golf.data_things.base_class - INFO - Current tourney ID: 401056527


In [ ]:
cmon.r

In [9]:
cmon.tourney_id

'401056527'

In [10]:
gs = dg.data_things.GameScoreboard()

2019-04-24 22:08:35,408 - donkey_golf.data_things.base_class - DEBUG - ABOUT TO SET yaml_sql_dict
2019-04-24 22:08:35,420 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-04-24 22:08:35,719 - donkey_golf.data_things.base_class - INFO - Current tourney ID: 401056527


In [11]:
gs.run()

2019-04-24 22:08:41,047 - donkey_golf.data_things.game_scoreboard - INFO - Pulling aggregated team score for scoreboard
2019-04-24 22:08:41,048 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-04-24 22:08:41,343 - donkey_golf.data_things.game_scoreboard - INFO - Determining where ties exist in the donkey scoreboard
2019-04-24 22:08:41,359 - donkey_golf.data_things.game_scoreboard - INFO - Adding ties to scoreboard data


In [12]:
gs.data

,id,username,donkey_score,missed_cut,player_left,rank,final_rank
0,7,None,-48,0,0,1,T1
1,9,None,-48,0,0,1,T1
2,6,None,-26,1,0,3,3
3,1,cloud,-17,1,0,4,4
4,5,None,-13,1,0,5,5


In [13]:
lb = dg.data_things.PullLeaderboard(user_id=1)

2019-04-24 22:09:37,403 - donkey_golf.data_things.base_class - DEBUG - ABOUT TO SET yaml_sql_dict
2019-04-24 22:09:37,415 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-04-24 22:09:37,725 - donkey_golf.data_things.base_class - INFO - Current tourney ID: 401056527


In [14]:
lb.run()

2019-04-24 22:09:42,447 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-04-24 22:09:42,706 - donkey_golf.data_things.tourney_leaderboard - INFO - Tourney current status: finished
2019-04-24 22:09:42,707 - donkey_golf.data_things.tourney_leaderboard - INFO - Pulling users team
2019-04-24 22:09:42,708 - donkey_golf.data_things.tourney_leaderboard - INFO - Pulling users active tourney team
2019-04-24 22:09:42,709 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-04-24 22:09:42,978 - donkey_golf.data_things.tourney_leaderboard - INFO - Tourney current status: finished
2019-04-24 22:09:42,979 - donkey_golf.data_things.tourney_leaderboard - INFO - Pulling users team
2019-04-24 22:09:42,979 - donkey_golf.data_things.tourney_leaderboard - INFO - Pulling users active tourney team
2019-04-24 22:09:42,980 - donkey_golf.data_things.base_class - INFO - Executing run_sql


In [16]:
tp = dg.data_things.TeamPlayerResults()

2019-04-24 22:10:47,073 - donkey_golf.data_things.base_class - DEBUG - ABOUT TO SET yaml_sql_dict
2019-04-24 22:10:47,086 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-04-24 22:10:47,362 - donkey_golf.data_things.base_class - INFO - Current tourney ID: 401056527


In [20]:
'YErp'.lower()

'yerp'

In [18]:
tp.pull_data()

2019-04-24 22:11:11,383 - donkey_golf.data_things.team_player_results - INFO - Pulling full team results by player
2019-04-24 22:11:11,384 - donkey_golf.data_things.base_class - INFO - Executing run_sql


In [19]:
tp.data

,id,username,pos,player,to_par,r1,r2,r3,r4,tot,earnings,fedex_pts,tourney_id,donkey_score,missed_cut,player_left,team_count
0,5,None,T5,Francesco Molinari,-11,70.0,67.0,66,74,277.0,"$403,938.00",106.0,401056527,-11,0.0,0.0,1.0
1,5,None,T2,Xander Schauffele,-12,73.0,65.0,70,68,276.0,"$858,667.00",230.0,401056527,-12,0.0,0.0,1.0
2,5,None,-,Paul Casey,CUT,81.0,73.0,--,--,154.0,--,0.0,401056527,9,1.0,0.0,1.0
3,5,None,T58,Takumi Kanaya (a),+5,73.0,74.0,68,78,293.0,--,0.0,401056527,5,0.0,0.0,1.0
4,5,None,T32,Viktor Hovland (a),-3,72.0,71.0,71,71,285.0,--,0.0,401056527,-3,0.0,0.0,2.0
5,5,None,T43,Li Haotong,-1,72.0,74.0,73,68,287.0,"$44,850.00",0.0,401056527,-1,0.0,0.0,1.0
6,6,None,-,Justin Rose,CUT,75.0,73.0,--,--,148.0,--,0.0,401056527,9,1.0,0.0,2.0
7,6,None,T2,Dustin Johnson,-12,68.0,70.0,70,68,276.0,"$858,667.00",230.0,401056527,-12,0.0,0.0,4.0
8,6,None,T21,Rory McIlroy,-5,73.0,71.0,71,68,283.0,"$107,956.00",42.0,401056527,-5,0.0,0.0,2.0
9,6,None,T21,Si Woo Kim,-5,72.0,72.0,70,69,283.0,"$107,956.00",42.0,401056527,-5,0.0,0.0,2.0


In [ ]:
yay = dg.data_things.TeamPlayerResults()

In [ ]:
yay.run()

In [ ]:
yay.clean_data()

In [ ]:
yay.data

In [ ]:
class TeamPlayerResults(dg.data_things.BaseClass):

    def pull_sql(self):
        '''
        Pulls full team results by player
        '''
        logger.info("Pulling full team results by player")
        sql = self.yaml_sql_dict.get('scoreboard_users_team')

        df = self.run_sql(sql)

        self.data = df.copy()

In [ ]:
cmon = dg.data_things.PullLeaderboard(user_id=5)

In [ ]:
cmon.run()

In [ ]:
cmon.data.loc[cmon.data['on_team'] == 'Yes']

In [ ]:
cmon.data.loc[cmon.data['on_team'] == 'Yes']['donkey_score'].astype(int).sum()

In [ ]:
BaseClass = dg.data_things.BaseClass

In [ ]:
class GameScoreboard(BaseClass):
    def __init__(self):
        BaseClass.__init__(self)

    def aggregate_user_scores(self):
        '''
        Pull each users aggregate score for the current tourney
        '''
        logger.info("Pulling aggregated team score for scoreboard")
        sql = self.yaml_sql_dict.get('aggregate_team_score')
        df = self.run_sql(sql)

        df.sort_values('donkey_score', inplace=True)

        df['rank'] = df['donkey_score'].rank(ascending=True, method='min').astype(int)

        self.data = df.copy()

    def determine_ties_in_scoreboard(self):
        '''
        Takes a scoreboard df and determines rank and handles tiesself.

        It's nice to have T2 on the scoreboard instead of multiple "2"s
        '''
        logger.info("Determining where ties exist in the donkey scoreboard")
        df_ties = self.data.copy()

        # Need to create a better rank that handles ties
        df_ties['new_rank'] = df_ties['rank'].copy()

        df_ties = df_ties.groupby(['rank'])['id'].count().reset_index()
        df_ties.loc[df_ties['id'] > 1, 'new_rank'] = "T"

        df_ties.loc[df_ties['new_rank'].notnull(), 'final_rank'] = 'T' + df_ties['rank'].astype(str)
        df_ties.loc[df_ties['new_rank'].isnull(), 'final_rank'] = df_ties['rank'].astype(str)

        self.df_ties = df_ties[['rank','final_rank']]

    def add_ties_to_data(self):
        logger.info("Adding ties to scoreboard data")
        self.data = self.data.merge(self.df_ties,
                                   how='left',
                                   on='rank')

    def run(self):
        self.aggregate_user_scores()
        self.determine_ties_in_scoreboard()
        self.add_ties_to_data()

In [ ]:
sb = GameScoreboard()

In [ ]:
sb.aggregate_user_scores()

In [ ]:
sb.data

In [ ]:
big = dg.data_things.TeamPlayerResults()

In [ ]:
big.run()

In [ ]:
big.data['player']

In [ ]:
big.user_id

In [ ]:
big.user_df

In [ ]:
big.run()

In [ ]:
big.

In [ ]:
big.determine_tourney_status()

In [ ]:
big.tourney_status

In [ ]:
yay.run()

In [ ]:
yay.data_dict

In [ ]:
yay.data

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(yay.data_dict)

In [ ]:
yay.tourney_id

In [ ]:
yay.load_table_to_db()

In [ ]:
from bs4 import BeautifulSoup
import requests

def current_tourney_data(url):
    source = requests.get(url).text
    espn = BeautifulSoup(source, 'lxml')
    tournament_info = espn.find('div', class_='Leaderboard__Header')
    tourney_data = {}
    course_data = tournament_info.find('div', class_= 'Leaderboard__Course__Location__Detail n8 clr-gray-04').text
    tourney_data['tourney_name'] = tournament_info.find('h1', class_='headline__h1 Leaderboard__Event__Title').text
    tourney_data['tourney_dates'] = tournament_info.find('span', class_='Leaderboard__Event__Date n7').text
    tourney_data['course_name'] = course_name = tournament_info.find('div', class_= 'Leaderboard__Course__Location n8 clr-gray-04').text
    tourney_data['course_par'] = course_data[3:5]
    tourney_data['course_yardage'] = course_data[-4:]
    tournament_winner_data = tournament_info.find('div', class_= 'n7 clr-gray-04')
    winner = []
    for node in tournament_winner_data.find_all('span', class_= 'clr-gray-05 mr2 label'):
        winner.append(node.next_sibling)
    tourney_data['tourney_purse'] = winner[0]
    tourney_data['tourney_defending_champ'] = winner[1]
    return tourney_data

df = current_tourney_data('http://www.espn.com/golf/leaderboard/_/tournamentId/401025263')

In [ ]:
df

In [ ]:
yay = dg.data_things.PullLeaderboard()

In [ ]:
yay.run()

In [ ]:
yay.calculate_cut_line()

In [ ]:
yay.determine_current_round()

In [ ]:
yay.run()

In [ ]:
big = dg.data_things.PullLeaderboard(user_id=5)

In [ ]:
big.run()

In [ ]:
big.user_df

In [ ]:
import oyaml
import pandas as pd
import logging
import psycopg2

from sqlalchemy import create_engine

from donkey_golf import config

# create logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

class BaseClass():

    def __init__(self, **kwargs):
        logger.debug("ABOUT TO SET yaml_sql_dict")
        self.conf = config.DGConfig()
        self.yaml_sql_dict = oyaml.load(open(self.conf.yaml_sql_loc), Loader=oyaml.FullLoader)
        self.tourney_id = self.determine_current_tourney_id()

    def run_sql(self, sql):
        '''
        Takes a SQL and connects to database to execute passed SQL
        '''
        #engine = create_engine(self.conf.db_url)
        with psycopg2.connect(self.conf.db_url) as conn:
        #with engine.connect() as conn:
            df = pd.read_sql(sql, conn)

        conn.close()

        return df

    def load_table_to_db(self, df, tablename):
        '''
        Takes a df and tablename and loads it to our database
        '''
        engine = create_engine(self.conf.db_url)

        with engine.connect() as conn:
            df.to_sql(tablename, conn, if_exists='replace', index=False)

    def determine_current_tourney_id(self):
        '''
        Pull each users aggregate score for the current tourney
        '''
        sql = self.yaml_sql_dict.get('determine_current_tourney_id')
        df = self.run_sql(sql)

        return df['value'][0].strip()


In [ ]:
class LoadWorldRankings(BaseClass):
    def __init__(self):
        BaseClass.__init__(self)

    def scrape_world_rankings_data(self):
        logger.info("Scraping world rankings data")
        rankings = pd.read_html('http://www.owgr.com/ranking?pageNo=1&pageSize=All&country=All', header=0)[0]

        keep_cols_rankings = ['This Week', 'Last week', 'End 2018', 'Name', 'Events Played (Actual)']
        rankings = rankings[keep_cols_rankings]
        
        self.data = rankings.copy()
        
    def clean_data(self):
        logger.info("Cleaning data")
        self.data.rename(columns= {
                                'This Week': 'current_rank',
                                'Last week':'lw_rank',
                                'End 2018':'ly_rank',
                                'Name':'player_raw',
                                'Events Played (Actual)':'events_played'},
                                inplace=True)
        self.fix_player_names()
        self.limit_to_top_players()
    
    def fix_player_names(self):
        logger.info("Fixing some players names that dont align between systems")

        # Some of the players names are messed up
        # read correct mappings from YAML file and apply them
        yaml_dict = oyaml.load(open(self.conf.yaml_world_rankings_loc), Loader=oyaml.FullLoader)
        player_map = yaml_dict.get('world_rankings')
        
        self.data['player_cleaned'] = self.data['player_raw'].map(player_map)
        self.data['player'] = self.data['player_cleaned'].combine_first(self.data['player_raw'])

    def limit_to_top_players(self):
        logger.info("Trimming world rankings to only top 400")
        self.data = self.data.loc[self.data['current_rank'] <= 400]
        
    def run(self):
        self.scrape_world_rankings_data()
        self.clean_data()
        self.load_table_to_db(self.data, 'pre_tourney')

In [ ]:
draft = LoadWorldRankings()

In [ ]:
draft.run()

In [ ]:
df = draft.data

In [ ]:
df.head(2)

In [ ]:
df.shape

In [ ]:
df.loc[df['current_rank'] < 400]

In [ ]:
draft.data

In [ ]:
cmon = dg.data_things.PullLeaderboard(user_id=5)

In [ ]:
cmon.pull_tourney_leaderboard()

In [ ]:
cmon.data

In [ ]:
import oyaml
import pandas as pd
import logging
import psycopg2

from sqlalchemy import create_engine

from donkey_golf import config


# create logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

class BaseClass():

    def __init__(self, **kwargs):
        logger.debug("ABOUT TO SET yaml_sql_dict")
        self.conf = config.DGConfig()
        self.yaml_sql_dict = oyaml.load(open(self.conf.yaml_sql_loc), Loader=oyaml.FullLoader)
        self.tourney_id = self.determine_current_tourney_id()

    def run_sql(self, sql):
        '''
        Takes a SQL and connects to database to execute passed SQL
        '''
        #engine = create_engine(self.conf.db_url)
        with psycopg2.connect(self.conf.db_url) as conn:
        #with engine.connect() as conn:
            df = pd.read_sql(sql, conn)

        conn.close()

        return df

    def load_table_to_db(self, df, tablename):
        '''
        Takes a df and tablename and loads it to our database
        '''
        engine = create_engine(self.conf.db_url)

        with engine.connect() as conn:
            df.to_sql(tablename, conn, if_exists='replace', index=False)

    def determine_current_tourney_id(self):
        '''
        Pull each users aggregate score for the current tourney
        '''
        sql = self.yaml_sql_dict.get('determine_current_tourney_id')
        df = self.run_sql(sql)

        return df['value'][0].strip()

In [ ]:
class GameScoreboard(BaseClass):
    def __init__(self):
        BaseClass.__init__(self)

    def aggregate_user_scores(self):
        '''
        Pull each users aggregate score for the current tourney
        '''
        logger.info("Pulling aggregated team score for scoreboard")
        sql = self.yaml_sql_dict.get('aggregate_team_score')
        df = self.run_sql(sql)

        df.sort_values('donkey_score', inplace=True)

        df['rank'] = df['donkey_score'].rank(ascending=True, method='min').astype(int)

        self.data = df.copy()
        
    def determine_ties_in_scoreboard(self):
        '''
        Takes a scoreboard df and determines rank and handles tiesself.

        It's nice to have T2 on the scoreboard instead of multiple "2"s
        '''
        logger.info("Determining where ties exist in the donkey scoreboard")
        df_ties = self.data.copy()

        # Need to create a better rank that handles ties
        df_ties['new_rank'] = df_ties['rank'].copy()

        df_ties = df_ties.groupby(['rank'])['id'].count().reset_index()
        df_ties.loc[df_ties['id'] > 1, 'new_rank'] = "T"

        df_ties.loc[df_ties['new_rank'].notnull(), 'final_rank'] = 'T' + df_ties['rank'].astype(str)
        df_ties.loc[df_ties['new_rank'].isnull(), 'final_rank'] = df_ties['rank'].astype(str)

        self.df_ties = df_ties[['rank','final_rank']]        
    
    def add_ties_to_data(self):
        logger.info("Adding ties to scoreboard data")
        self.data = self.data.merge(self.df_ties,
                                   how='left',
                                   on='rank')
        
    def run(self):
        self.aggregate_user_scores()
        self.determine_ties_in_scoreboard()
        self.add_ties_to_data()

In [ ]:
gs = GameScoreboard()

In [ ]:
gs.run()

In [ ]:
gs.data

In [ ]:
gs.aggregate_user_scores()

In [ ]:
gs.data

In [ ]:
gs.determine_ties_in_scoreboard()

In [ ]:
gs.add_ties_to_data()

In [ ]:
from donkey_golf import data_utils as du

In [ ]:
du.pull_scoreboard()

In [ ]:
class Leaderboard(StockPull):
    
    def __init__(self):
        StockPull.__init__(self)
        self.determine_current_url()
    
    def determine_current_url(self):
        '''
        Returns the specific URL we want to scrape from
        '''        
        self.url_lb = 'http://www.espn.com/golf/leaderboard/_/tournamentId/{}'.format(self.tourney_id)
    
    def scrape_espn_leaderboard(self, url):
        '''
        Scrape the main leaderboard table from the web
        '''
        data_dict = {}
        # Creating leaderboard & rankings dataframes
        leaderboard = pd.read_html(url,header=3)[0]
        leaderboard.columns = leaderboard.columns.str.lower().str.replace(' ','_')

        leaderboard['tourney_id'] = self.tourney_id
        
        self.data = leaderboard.copy()
    
    def determine_tourney_status(self):
        '''
        Look at the leaderboard data and determine current tourney
        status from that data
        '''
        if 'tee_time' in self.data.columns:
            self.tourney_status = 'pre_tourney'
        elif 'fedex_pts' in self.data.columns or 'earnings' in self.data.columns:
            self.tourney_status = 'finished'
            
        elif 'r4' in self.data.columns or 'tot' in self.data.columns:
            self.tourney_status = 'in_progress'
        else:
            self.tourney_status = 'cant_determine'
            
        logger.info(f"Tourney current status: {self.tourney_status}")
        
    def determine_current_round(self):
        '''
        Figure out which round is in progress. Will be helpful for cut
        calculations among other things
        '''
        logger.info("Determining the current round")
        df = self.data.copy()
        
        df = df.loc[df['missed_cut'] == 0]
        
        current_round = None
        
        if 'fedex_pts' in df.columns:
            current_round = 'finished'
        else:
            rounds = ['r1','r2','r3','r4']
            for round_num in rounds:
                if df.loc[df[round_num] == '--'].shape[0] > 1:
                    current_round = round_num                    
        logger.info(f"Determined the current round to be {current_round}")

        return current_round
        
    def add_donkey_game_score(self):
        '''
        Will add a column called donkey score that should
        convert the to_par string to an integer
        '''
        logger.info("Calculating the donkey_score column")
        self.data.loc[self.data['to_par'] == 'E', 'donkey_score'] = 0
        self.data.loc[self.data['to_par'].str[0:1] == '+', 'donkey_score'] = self.data['to_par'].str[1:]
        self.data.loc[self.data['to_par'].str[0:1] == '-', 'donkey_score'] = self.data['to_par']

        # IF they don't have a score of E or one that starts with a '+' or '-'
        # let's assume they didn't make it to the weekend
        self.add_score_for_cut_players()
    
    def add_score_for_cut_players(self):
        '''
        Determines the score for cut players
        and adds a missed_cut field
        '''
        logger.info("Determining score for cut players and adding missed_cut indicator")
        cut_score = self.data.loc[self.data['donkey_score'].notnull(), 'donkey_score'].astype(int).max()
        cut_score = cut_score + 1

        self.data['missed_cut'] = 0
        self.data.loc[self.data['donkey_score'].isnull(), 'missed_cut'] = 1

        self.data.loc[self.data['donkey_score'].isnull(), 'donkey_score'] = cut_score
        
    def add_player_left_indicator(self):
        '''
        This should add a column 'player_left'
        indicating if a player is still in progress
        '''
        logger.info("Determine if the player is active and if he has been cut or not")
        self.data['player_left'] = 1

        self.data.loc[self.data['missed_cut'] == 1, 'player_left'] = 0

        self.data.loc[(self.data['missed_cut'] == 0) &
              (self.data['r4'] != '--' ),
              'player_left'] = 0

    def add_golfer_team_count(self):
        '''
        This should add a column 'team_count'
        indicating the amount of teams a player is on
        '''
        self.golfer_team_count()
        
        logger.debug("Adding golfer team count")
        self.data = self.data.merge(self.golfer_count_by_team,
            how='left',
            on='player')

        self.data['team_count'] = self.data['team_count'].fillna(0.0).astype(int)

    def golfer_team_count(self):
        '''
        Pull each users aggregate score for the current tourney
        '''
        sql = yaml_sql_dict.get('golfer_team_count')
        df = self.run_sql(sql)

        self.golfer_count_by_team = df
    
    def prepare_active_data(self):
        '''
        If we know the data is from after the tourney has started,
        there will be manipulations we need to do
        '''
        logger.info("Running prepare_in_progress_data")
        self.clean_active_cols()
        self.add_donkey_game_score()
        self.add_player_left_indicator()
        self.add_golfer_team_count()

    def clean_active_cols(self):
        '''
        Clean function for the active tourney data
        '''
        self.data = self.data.drop('unnamed:_1', axis=1)
    
    def prepare_pre_tourney_data(self):
        '''
        This will be used if we need to make any updated to the pre_tourney data
        '''
        pass
    
    def clean_data(self):  
        '''
        Depending on the status of the tourney (and what .data will look like)
        we want to prepare/cleanse the data
        '''
        logger.info("Running clean_data function")
        if self.tourney_status in ['in_progress','finished']:
            self.prepare_active_data()
        elif self.tourney_status == 'pre_tourney':
            self.prepare_pre_tourney_data()
            
    def load_leaderboard_to_db(self):
        '''
        Take self.data and load it into the appropriate table based
        on the tourney_status
        '''
        if self.tourney_status == 'pre_tourney':
            logger.info("Loading data into the pre_tourney table")
            self.load_table_to_db(self.data, 'pre_tourney')
        else:
            logger.info("Loading data into the leaderboard table")            
            self.load_table_to_db(self.data, 'leaderboard')
        
    def run(self):
        # Get the leaderboard data
        self.scrape_espn_leaderboard(self.url_lb)
        
        # Determine the tourney status (set an attribute)
        self.determine_tourney_status()
        
        # Clean data - smart enough to determine pre_tourney or in-progress data
        self.clean_data()
        
        # Don't need this right now
        #self.load_leaderboard_to_db()

In [ ]:
fox = Leaderboard()

In [ ]:
fox.run()

In [ ]:
fox.determine_current_round()

In [ ]:
fox.load_leaderboard_to_db()

In [ ]:
fox.run()

In [ ]:
fox.data

In [ ]:
fox.load_leaderboard_to_db()

In [ ]:
fox.data

In [ ]:
fox.determine_current_round()

In [ ]:
fox.data

In [ ]:
class PullLeaderboard(StockPull):
    
    def __init__(self, **kwargs):
        StockPull.__init__(self)
        self.kwargs = kwargs
        if kwargs.get('user_id'):
            self.user_id = kwargs.get('user_id')
        else:
            self.user_id = None
            
    def pull_tourney_leaderboard(self):
        '''
        Pulls leaderboard data for the tourney
        '''

        sql = yaml_sql_dict.get('pull_tourney_leaderboard')
        self.data = self.run_sql(sql)
        self.determine_tourney_status()
        
        if self.user_id and self.tourney_status == 'in_progress':
            self.pull_users_team()
            
            # If the user_df comes back with records, add on_team field
            if not self.user_df.empty:
                self.data = self.data.merge(user_df[['player','id']],
                    how='left',
                    on=['player'])

                self.data['on_team'] = 'No'
                self.data.loc[self.data['id'].notnull(), 'on_team'] = 'Yes'

                self.data = self.data.drop('id', axis=1)

        self.data['team_count'] = self.data['team_count'].astype(int)
    
    def determine_tourney_status(self):
        '''
        Look at the leaderboard data and determine current tourney
        status from that data
        '''
        if 'tee_time' in self.data.columns:
            self.tourney_status = 'pre_tourney'
        elif 'fedex_pts' in self.data.columns or 'earnings' in self.data.columns:
            self.tourney_status = 'finished'
            
        elif 'r4' in self.data.columns or 'tot' in self.data.columns:
            self.tourney_status = 'in_progress'
        else:
            self.tourney_status = 'cant_determine'
            
        logger.info(f"Tourney current status: {self.tourney_status}")
        
    def pull_users_team(self):
        '''
        Pulls leaderboard data for everyone on a given users team
        '''
        
        if self.tourney_status == 'pre_tourney':
            logger.info("Pulling users pre_tourney team")
            sql = yaml_sql_dict.get('users_team_pre_tourney')
            sql = sql.format(self.user_id, conf.tourney_id)
            self.user_df = run_sql(sql)
        else:
            logger.info("Pulling users active tourney team")            
            sql = yaml_sql_dict.get('users_team')
            # Sub in users ID into the SQL
            sql = sql.format(self.user_id, self.tourney_id)
            self.user_df = self.run_sql(sql)

In [ ]:
yay = PullLeaderboard(user_id=2)

In [ ]:
yay.pull_tourney_leaderboard()

In [ ]:
yay.user_df

In [ ]:
yay.data

In [ ]:
yay.pull_tourney_leaderboard()

In [ ]:
yay.tourney_id

In [ ]:
yay.kwargs

In [ ]:
yay.user_id

In [ ]:
yay.pull_users_team()

In [ ]:
yay.pull_tourney_leaderboard()

In [ ]:
yay.data

In [ ]:
yay.pull_tourney_leaderboard()

In [ ]:
yay.data

In [ ]:
fox.tourney_status in ['in_progress','finished']

In [ ]:
fox.data

In [ ]:
df = fox.data

In [ ]:
df

In [ ]:
fox.add_donkey_game_score()

In [ ]:
fox.data

In [ ]:
df = fox.data

In [ ]:
'earnings' in fox.data.columns

In [ ]:
'fedex_pts' in fox.data.columns or 'earnings' in fox.data.columns

In [ ]:
fox.tourney_id

In [ ]:
fox.url_lb

In [ ]:
fox.scrape_espn_leaderboard(fox.url_lb)

In [ ]:
cd donkey_golf/

In [ ]:
ls

In [ ]:
cd all_things_data/

In [ ]:
import sql_classes